In [1]:
%run RaceCar.py
import numpy as np
import pygame

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def generate_map(w_cells, h_cells, min_lim, max_lim):
    race_map = np.zeros((w_cells, h_cells), dtype=np.int32)
    # Set boundaries
    race_map[w_cells // 2:, h_cells // 2:] = -1
    lims = np.random.randint(min_lim, max_lim, size=4)
    for h in range(h_cells):
        lims[0] = np.amax([min_lim, np.amin([max_lim, lims[0] + np.random.choice([-1, 0, 1])])])
        race_map[h, :lims[0]] = -1
        if h > h_cells // 2:
            lims[1] = np.amax([min_lim, np.amin([max_lim, lims[1] + np.random.choice([-1, 0, 1])])])
            race_map[h, h_cells // 2 - lims[1]:] = -1
    for w in range(w_cells):
        lims[2] = np.amax([min_lim, np.amin([max_lim, lims[2] + np.random.choice([-1, 0, 1])])])
        race_map[:lims[2], w] = -1
        if w > w_cells // 2:
            lims[3] = np.amax([min_lim, np.amin([max_lim, lims[3] + np.random.choice([-1, 0, 1])])])
            race_map[w_cells // 2 - lims[3]:, w] = -1
    # Set start line
    race_map[-1, np.argwhere(race_map[-1, :] == 0)] = 1
    # Set finish line
    race_map[np.argwhere(race_map[:, -1] == 0), -1] = 2
    return race_map

#
width = 850
height = 850
w_cells = 30
h_cells = 30
min_lim = 1
max_lim = 6

race_map = generate_map(w_cells, h_cells, min_lim, max_lim)


def func():
    while True:
        draw_map(screen, race_map)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        pygame.display.update()

In [8]:
np.argwhere(race_map == 1)

array([[29,  1],
       [29,  2],
       [29,  3],
       [29,  4],
       [29,  5],
       [29,  6],
       [29,  7],
       [29,  8],
       [29,  9],
       [29, 10],
       [29, 11]])

In [3]:
# Number of states.
S = race_map.shape[0] * race_map.shape[1]
init_states = np.argwhere(race_map == 1)
# Number of actions.
A = 66
# Init a randomic policy.
pi = np.random.randint(66, size=S)
# Discount factor.
gamma = 1.0
# Exploration degree.
epsilon = 0.1
# Number of episodes.
episodes = 100
# Inizialization.
N = np.zeros((S, A), dtype=np.int32)
Q = np.zeros((S, A), dtype=np.float64)

# Loop on episodes.
for e in range(episodes):
    # Exploring start.
    init_coords = init_states[np.random.randint(init_states.shape[0])]
    # Run a game.
    car = RaceCar(race_map, init_coords, pi, epsilon, width, height)
    car.run()
    states = car.get_states()
    actions = car.get_actions()
    rewards = car.get_rewards()
    G = 0.0
    for t, s_t in reversed(list(enumerate(states))):
        G = rewards[t] + gamma * G
        N[s_t, actions[t]] += 1
        Q[s_t, actions[t]] += (1.0 / N[s_t, actions[t]]) * (G - Q[s_t, actions[t]])
        pi[s_t] = np.argmax(Q[s_t, :])

880
878
872
881
877
874
614
586
884
793
871
555
885
826
795
707
883
876
875
615
886
889
740
879
739
799
527
615
466
500
555
886
713
886
830
873
496
499
410
563
736
800
740
713
802
533
438
740
591
590
530
409
465
559
168
626
442
407
559
779
446
112
411
409
476
319
654
629
596
563
899
349
413
324
147
869
749
749
567
354
294
119
689
59
563
775
749
869
629
599
745
479
749
115
87
685
599
569
719
866
